## **TESTING:** etl_21 -- TSDB

In [2]:
import os
import pandas as pd
import pyarrow
import pytz
from datetime import datetime, timedelta
import pyarrow.feather as feather
import numpy as np
pd.set_option('display.max_columns', None)
import matplotlib.pyplot as plt
from pandasql import sqldf
import sqlite3
import argparse
import shutil
import psycopg2
from sqlalchemy import create_engine


pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)


### **CONENCTION**

In [3]:
CONNECTION = "postgres://tsdbadmin:>X>#h2lWqXESlyGd}mj2NPDlB@esd7mq3z84.dts890uzaz.tsdb.cloud.timescale.com:37281/tsdb?sslmode=require"

Using psycopg2

In [4]:
conn = psycopg2.connect(CONNECTION)

Using SQLAlchemy

In [24]:
# conn = create_engine(CONNECTION)

Convert History to SCD

# **CHECKS** Scope

In [8]:
l_check_prices = ['61277.43000000',
 '61111.20000000',
 '61268.29000000',
 '61132.62000000',
 '61299.90000000',
 '61058.75000000',
 '61056.20000000',
 '61223.36000000',
 '61065.03000000',
 '61227.34000000',
 '61056.70000000',
 '61237.96000000',
 '61170.44000000',
 '61136.34000000',
 '61222.59000000',
 '61111.51000000',
 '61227.27000000',
 '61158.14000000',
 '61303.54000000',
 '61047.88000000']

# **CHECKS** Database

### Tables

In [6]:
import psycopg2

def list_tables(conn):
    # Create a cursor object
    cursor = conn.cursor()

    # Execute the query to list all tables
    query = """
    SELECT table_schema, table_name
    FROM information_schema.tables
    WHERE table_type = 'BASE TABLE' and table_schema = 'public'
    ORDER BY table_schema, table_name;
    """
    cursor.execute(query)

    # Fetch all the results
    tables = cursor.fetchall()

    # Print the results
    for table in tables:
        # print(table)
        print(f"{table[0]}.{table[1]}")

    # Close the cursor and connection
    cursor.close()
    # conn.close()

list_tables(conn)


public.l2_history_ask
public.l2_history_bid
public.l2_snapshot_ask
public.l2_snapshot_bid
public.logs


Convert History to SCD

### Snapshot

In [22]:
# Define the query
sql_s01 = 'SELECT * FROM l2_snapshot_ask'

# Execute the query and load the data into a pandas DataFrame
s01_01 = pd.read_sql_query(sql_s01, conn)
s01_02 = s01_01[s01_01['price'].isin(l_check_prices)].copy()

display(s01_02)

/tmp/ipykernel_49309/2670138738.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  s01_01 = pd.read_sql_query(sql_s01, conn)


,ticker,side,price,px_00,quantity,qx_00,timestamp,lastupdateid


### History

In [21]:
# Define the query
sql_h01 = 'SELECT * FROM l2_history_ask'

# Execute the query and load the data into a pandas DataFrame
h01_01 = pd.read_sql_query(sql_h01, conn)
h01_02 = h01_01[h01_01['price'].isin(l_check_prices)]
display(h01_02)

/tmp/ipykernel_49309/3255620334.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  h01_01 = pd.read_sql_query(sql_h01, conn)


,ticker,side,price,px_00,quantity,qx_00,timestamp,lastupdateid,scd_from_date,scd_to_date


# **CHECKS** Uniqueness

In [ ]:

    db_snap_02 = pd.read_sql_query(
        """
        SELECT  lastupdateid
        ,count(distinct side) as D_Side
        FROM l2_snapshot
        GROUP BY lastupdateid 
        """
        , conn)
    db_snap_02.D_Side.value_counts()    


# **CHECKS** Logs

In [28]:
# Connect to the SQLite database
# Define the query
sql_logs = 'SELECT * FROM logs'

# Execute the query and load the data into a pandas DataFrame
db_logs_01 = pd.read_sql_query(sql_logs, conn)

# Display the DataFrame
print(db_logs_01.level.value_counts())
print("---------------------------------------------------------------")
# display(db_logs_01)

level
INFO     9
ERROR    9
Name: count, dtype: int64
---------------------------------------------------------------


/tmp/ipykernel_49309/2746400065.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  db_logs_01 = pd.read_sql_query(sql_logs, conn)


In [29]:
display(db_logs_01.head())
display(db_logs_01.tail())

,timestamp,level,p_db_ver,message
0,2024-05-24 23:20:56.450389,INFO,20_load_snapshot.py,***main*** LOADING to Database
1,2024-05-24 23:20:56.501350,ERROR,20_load_snapshot.py,Error in main function: name 'p_db_ver' is not defined
2,2024-05-24 23:25:54.897077,INFO,20_load_snapshot.py,***main*** LOADING to Database
3,2024-05-24 23:25:54.928190,ERROR,20_load_snapshot.py,Error in main function: name 'p_db_ver' is not defined
4,2024-05-24 23:28:35.237079,INFO,20_load_snapshot.py,***main*** LOADING to Database


,timestamp,level,p_db_ver,message
13,2024-05-24 23:33:12.476985,ERROR,20_load_snapshot.py,Error in main function: name 'p_db_ver' is not defined
14,2024-05-24 23:33:34.681238,INFO,20_load_snapshot.py,***main*** LOADING to Database
15,2024-05-24 23:33:34.711472,ERROR,20_load_snapshot.py,Error in main function: name 'p_db_ver' is not defined
16,2024-05-24 23:36:46.255040,INFO,20_load_snapshot.py,***main*** LOADING to Database
17,2024-05-24 23:36:46.279400,ERROR,20_load_snapshot.py,Error in main function: f_ETL_Init_01() missing 1 required positional argument: 'conn'


In [30]:
db_logs_01

,timestamp,level,p_db_ver,message
0,2024-05-24 23:20:56.450389,INFO,20_load_snapshot.py,***main*** LOADING to Database
1,2024-05-24 23:20:56.501350,ERROR,20_load_snapshot.py,Error in main function: name 'p_db_ver' is not defined
2,2024-05-24 23:25:54.897077,INFO,20_load_snapshot.py,***main*** LOADING to Database
3,2024-05-24 23:25:54.928190,ERROR,20_load_snapshot.py,Error in main function: name 'p_db_ver' is not defined
4,2024-05-24 23:28:35.237079,INFO,20_load_snapshot.py,***main*** LOADING to Database
5,2024-05-24 23:28:35.267089,ERROR,20_load_snapshot.py,Error in main function: name 'p_db_ver' is not defined
6,2024-05-24 23:30:57.591233,INFO,20_load_snapshot.py,***main*** LOADING to Database
7,2024-05-24 23:30:57.624228,ERROR,20_load_snapshot.py,Error in main function: name 'p_db_ver' is not defined
8,2024-05-24 23:32:21.381040,INFO,20_load_snapshot.py,***main*** LOADING to Database
9,2024-05-24 23:32:21.413184,ERROR,20_load_snapshot.py,Error in main function: name 'p_db_ver' is not defined
